In [3]:
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# index 생성

In [5]:
from pinecone import Pinecone
# 클라이언트 생성
pinecone_client = Pinecone(api_key=PINECONE_API_KEY)

In [6]:
pinecone_client

In [ ]:
# 인덱스 create
from pinecone import ServerlessSpec
pinecone_client.create_index(
    name='embedding-3d',
    dimension=3,
    metric='cosine',
    spec= ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

{
    "name": "embedding-3d",
    "metric": "cosine",
    "host": "embedding-3d-2wxeig8.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 3,
    "deletion_protection": "disabled",
    "tags": null
}

In [10]:
index = pinecone_client.Index('embedding-3d')
index

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
index.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [ ]:
for ids in index.list(namespace='embedding-3d-ns1'): # default: namespace='embedding-3d-ns1
    print(ids)

In [15]:
# pinecone data 형식
[{'id': 'vec1', 'values' : [1.0, 1.5, 2.0], 'metadata' : {'genre' : 'drama'}},
 {'id': 'vec2', 'values' : [2.0, 1.5, 0.5], 'metadata' : {'genre' : 'action'}},
 {'id': 'vec3', 'values' : [0.1, 0.3, 0.5], 'metadata' : {'genre' : 'drama'}},
 {'id': 'vec4', 'values' : [1.0, 2.5, 3.0], 'metadata' : {'genre' : 'action'}},
 {'id': 'vec5', 'values' : [3.0, 1.2, 1.3], 'metadata' : {'genre' : 'action'}},
 {'id': 'vec6', 'values' : [0.3, 1.1, 2.5], 'metadata' : {'genre' : 'drama'}}
 ]

[{'id': 'vec1', 'values': [1.0, 1.5, 2.0], 'metadata': {'genre': 'drama'}},
 {'id': 'vec2', 'values': [2.0, 1.5, 0.5], 'metadata': {'genre': 'action'}},
 {'id': 'vec3', 'values': [0.1, 0.3, 0.5], 'metadata': {'genre': 'drama'}},
 {'id': 'vec4', 'values': [1.0, 2.5, 3.0], 'metadata': {'genre': 'action'}},
 {'id': 'vec5', 'values': [3.0, 1.2, 1.3], 'metadata': {'genre': 'action'}},
 {'id': 'vec6', 'values': [0.3, 1.1, 2.5], 'metadata': {'genre': 'drama'}}]

In [14]:
index.upsert(
    vectors=[{'id': 'vec1', 'values' : [1.0, 1.5, 2.0], 'metadata' : {'genre' : 'drama'}},
    {'id': 'vec2', 'values' : [2.0, 1.5, 0.5], 'metadata' : {'genre' : 'action'}},
    {'id': 'vec3', 'values' : [0.1, 0.3, 0.5], 'metadata' : {'genre' : 'drama'}},
    {'id': 'vec4', 'values' : [1.0, 2.5, 3.0], 'metadata' : {'genre' : 'action'}},
    {'id': 'vec5', 'values' : [3.0, 1.2, 1.3], 'metadata' : {'genre' : 'action'}},
    {'id': 'vec6', 'values' : [0.3, 1.1, 2.5], 'metadata' : {'genre' : 'drama'}}
    ],
    namespace='embedding-3d-ns1'
        
)# update _insert

{'upserted_count': 6}

In [16]:
index.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'embedding-3d-ns1': {'vector_count': 6}},
 'total_vector_count': 6,
 'vector_type': 'dense'}

In [ ]:
# id list
for ids in index.list(namespace='embedding-3d-ns1'): # default: namespace='embedding-3d-ns1
    print(ids)

['vec1', 'vec2', 'vec3', 'vec4', 'vec5', 'vec6']


# query

In [20]:
response = index.query(
    namespace='embedding-3d-ns1',
    vector=[0.1,0.4,0.7],
    top_k=3,
    include_values=True,
    include_metadata=True,
    filter={'genre': {'$eq':'drama'}}
)
response

{'matches': [{'id': 'vec3',
              'metadata': {'genre': 'drama'},
              'score': 1.00008953,
              'values': [0.1, 0.3, 0.5]},
             {'id': 'vec6',
              'metadata': {'genre': 'drama'},
              'score': 0.99550879,
              'values': [0.3, 1.1, 2.5]},
             {'id': 'vec1',
              'metadata': {'genre': 'drama'},
              'score': 0.961182,
              'values': [1.0, 1.5, 2.0]}],
 'namespace': 'embedding-3d-ns1',
 'usage': {'read_units': 1}}